<a href="https://colab.research.google.com/github/Darkdew/Web-scrapper-jsload-dynamic/blob/main/std_scrape_filtered_code_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
url = 'https://patient.info/forums/discuss/browse/genital-herpes-simplex-979'

In [ ]:
r = requests.get(url, headers= headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
thread_name = soup.find_all('div',{'class':'avt avt-sm'})

In [ ]:
# next_page = soup.find_all('select',{'class':'submit reply__control reply-pagination'})[0].find('option')
next_page = soup.find_all('div',{'class':'group-discussions__paginate_bottom'})[0].find('option').text

In [ ]:
page_count = int(next_page.split('/')[1])

In [ ]:
import pandas as pd

thread_name_list=[]
thread_starter_name_list=[]
thread_url_list=[]
# reply_url_list = []
profile_url_list=[]
number_of_reply_list =[]
last_teply_at_list=[]
full_list=[]
no_of_replies_list=[]
thread_content_list=[]
likes_count_list=[]

next_page = soup.find_all('div',{'class':'group-discussions__paginate_bottom'})[0].find('option').text
page_count = int(next_page.split('/')[1])
j=0
attempt=0
while attempt<5:
  try:
    
    for page_cnt in range(j, page_count):
        
        main_url= 'https://patient.info/forums/discuss/browse/genital-herpes-simplex-979'
        current_url = main_url+"?page=%d" %(page_cnt)+'#group-discussions'
        print(current_url)
        r_current_url = requests.get(current_url, headers= headers)
        current_url_soup = BeautifulSoup(r_current_url.text, 'html.parser')
        current_url_thread_name = current_url_soup.find_all('div',{'class':'avt avt-sm'})
              
        for item in current_url_thread_name:
            temp_list=[]
            # Extracting thread title
            """
            title1_a_tag=item.find('h3',{'class':'post__title'}).find('a')
            strings = str(title1_a_tag)
            soup = BeautifulSoup(strings,'html.parser')
            thread_name_list.append(soup.find('a').encode_contents().decode('utf_8'))
            """
            title1_a_tag=item.find('h3',{'class':'post__title'}).find('a').text
            thread_name_list.append(title1_a_tag)
            
            # extracting thread starter name
        #     profile_name=item.find('a',{'class':'action-link'}).text  ## we can get profile name by this line of code as well 
            profile_name=item.find('a',{'aria-label':'profile'}).text
            profile_name_cleaned = profile_name.replace('\r\n','').replace(" ","")
            thread_starter_name_list.append(profile_name_cleaned)
            
            # extracting thread url
            # thread_url = item.find('a')['href']  #this line of code is also working to get the profile url
            thread_url = item.find('h3',{'class':'post__title'}).find('a')['href']
            thread_url_complete = "https://patient.info" + thread_url
            #thread_url_list.append(thread_url_complete)
            
            ## extracting the content of the Thread URL
            thread_content_r1 = requests.get(thread_url_complete, headers= headers)
            thread_content_soup = BeautifulSoup(thread_content_r1.text, 'html.parser')
            thread_content_temp = thread_content_soup.find_all('div',{'id':'post_content'})[0].find('input')['value']
            thread_content_temp1 = thread_content_temp.replace('\r','').replace('\n','')
            thread_content= BeautifulSoup(thread_content_temp1,'lxml').get_text()
            
            
            
        #     thread_content_list.append(thread_content)
            ## extracting urls of replies however this is same as thread_url extracted above
            """
            reply_url = item.find('a',{'title':'View replies'})['href']
            reply_url_list.append(reply_url)
            """
            
            ## Extracting urls of profiles
            profile_url = item.find('a',{'aria-label':'profile'})['href']
            profile_url_complete = "https://patient.info" + profile_url
            profile_url_list.append(profile_url_complete)
            
            last_teply_at = item.find('span',{'class':'last-reply-on'}).find('time')['datetime']
            last_teply_at_list.append(last_teply_at)
            
            no_of_replies = item.find('div',{'class':'post__actions post__discussions likes_replies grid container'}).find('span',"position"==2).text
            no_of_replies_list.append(no_of_replies)
            
            no_of_likes= item.find('span',{'class':'last-reply-on'}).text
            
            #iterating on total number of replies for each thread on master page
            """for i in range(no_of_replies):
              ##
              response_text = soup.find_all('div',{'class':'post__content break-word'})[i].find('input')['value']
              
              
              ##
              response_author_name= soup.find_all('h5',{'class':'author__info'})[i].find('a').text
              

              ##
              response_recipient_name= soup.find_all('a',{'class':'author__recipient'})[i].text


              ##
              response_timestamp =soup.find_all('time',{'class':'fuzzy'})[i]['datetime']
              #
        #         process replies on next page
            """
            temp_list=[title1_a_tag, profile_name_cleaned, no_of_replies,last_teply_at,thread_url_complete,thread_content]
            full_list.append(temp_list)
        j=j+1
          
    break
  
  except Exception as e:
    print(e)
    print(j)
    attempt= attempt+1
df = pd.DataFrame(full_list,columns=("Question","Postby","Number of Replies","Last Reply At","Thread Url","Content"))

In [ ]:
df.head()

,Question,Postby,Number of Replies,Last Reply At,Thread Url,Content
0,Request for Forum User Feedback,Emis_Moderator,1,2021-04-20T04:08+00:00,https://patient.info/forums/discuss/request-fo...,"Hello,We recognise that recent changes to the ..."
1,Genital Herpes: Useful info for those looking ...,FelizCastus,535,2020-07-09T14:38+00:00,https://patient.info/forums/discuss/genital-he...,Ever since I suspected genital herpes (GH) and...
2,Is this an outbreak of genital hsv on upper th...,parsnipz,2,2021-05-29T17:10+00:00,https://patient.info/forums/discuss/is-this-an...,I found out years ago I was exposed to genital...
3,Anal discomfort/discharge,ann14932,5,2021-05-29T16:35+00:00,https://patient.info/forums/discuss/anal-disco...,I have been having some anal discomfort for a ...
4,Is this herpes ?,kate45354,3,2021-05-29T10:36+00:00,https://patient.info/forums/discuss/is-this-he...,ive been having bad yeast infections back to b...


In [ ]:
len(df)

10099

In [ ]:
df.to_csv('/content/sample_data/master_df.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df_test = df.loc[[28]]
#df_test

In [ ]:
full_final_list=[]
# import pandas as pd

for index, row in df.iterrows():
    temp_list1 =[]
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    
    if int(row['Number of Replies'])>32:
        page_count = 0
        
        while page_count < 34:
            strings= row['Thread Url']
            url = strings+"?page=%d" %(page_count)
            print(url)
            
            r = requests.get(url, headers= headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            
            response_timestamp =soup.find_all('time',{'class':'fuzzy'})#[i]['datetime']
            print(len(response_timestamp))
      
            for i in range(1,len(response_timestamp)):
                response_author_name= soup.find_all('h5',{'class':'author__info'})[i].find('a').text
                response_timestamp =soup.find_all('time',{'class':'fuzzy'})[i]['datetime']
                
                if soup.find('p',{'data-type':'deleted'}) or soup.find('p',{'data-type':'moderation'}):
                    response_text = soup.find_all('div',{'class':'post__content break-word'})[i-1].find('p').text
                else:
                    response_text = soup.find_all('div',{'class':'post__content break-word'})[i-1].find('input')['value']
                    
                thread_content_temp1 = thread_content_temp.replace('\r','').replace('\n','')
                thread_content= BeautifulSoup(thread_content_temp1,'lxml').get_text()
                
                #print(response_author_name)
#                 print(response_text)
                temp_list1= [row['Question'], row['Postby'], row['Number of Replies'], row['Last Reply At'], row['Thread Url'], row['Content'],response_author_name, response_timestamp, response_text]
                full_final_list.append(temp_list1)
            page_count += 1
    else:
        url= row['Thread Url']
        r = requests.get(url, headers= headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        response_timestamp =soup.find_all('time',{'class':'fuzzy'})#[i]['datetime']
        print(len(response_timestamp))
        print(url)
        #for i in range(5):
        for i in range(1,len(response_timestamp)):
            response_author_name= soup.find_all('h5',{'class':'author__info'})[i].find('a').text
            response_timestamp =soup.find_all('time',{'class':'fuzzy'})[i]['datetime']
            if soup.find('p',{'data-type':'deleted'}) or soup.find('p',{'data-type':'moderation'}):
                response_text= 'Message Deleted/Adjusted by moderator'
                #response_text = soup.find_all('div',{'class':'post__content break-word'})[i-1].find('p').text
            else:
                response_text = soup.find_all('div',{'class':'post__content break-word'})[i-1].find('input')['value']
      #print(response_author_name)
#             print(response_text)
            temp_list1= [row['Question'], row['Postby'], row['Number of Replies'], row['Last Reply At'], row['Thread Url'], row['Content'],response_author_name, response_timestamp, response_text]
            full_final_list.append(temp_list1)
  ##
  #temp_list1= [row['Question'], row['Postby'], row['Number of Replies'], row['Last Reply At'], row['Thread Url'], row['Content'],response_author_name, response_timestamp]#, response_text]
  #full_final_list.append(temp_list1)
df_final = pd.DataFrame(full_final_list,columns=("Master_Question","Master_Postby","Master_Number of Replies","Master_Last Reply At","Master_Thread Url","Master_Content","response_author_name","response_timestamp","response_text"))

In [ ]:
#pd.set_option('max_colwidth',None)
df_final.head()

,Master_Question,Master_Postby,Master_Number of Replies,Master_Last Reply At,Master_Thread Url,Master_Content,response_author_name,response_timestamp,response_text
0,Request for Forum User Feedback,Emis_Moderator,1,2021-04-20T04:08+00:00,https://patient.info/forums/discuss/request-fo...,"Hello,We recognise that recent changes to the ...",AnnaMarie73,2021-05-26T00:50+00:00,Message Deleted/Adjusted by moderator
1,Request for Forum User Feedback,Emis_Moderator,1,2021-04-20T04:08+00:00,https://patient.info/forums/discuss/request-fo...,"Hello,We recognise that recent changes to the ...",mat44954,2021-05-09T10:38+00:00,Message Deleted/Adjusted by moderator
2,Genital Herpes: Useful info for those looking ...,FelizCastus,535,2020-07-09T14:38+00:00,https://patient.info/forums/discuss/genital-he...,Ever since I suspected genital herpes (GH) and...,FelizCastus,2021-05-03T16:04+00:00,[b]Genital HSV symptoms[/b]\r\n\r\nEveryone kn...
3,Genital Herpes: Useful info for those looking ...,FelizCastus,535,2020-07-09T14:38+00:00,https://patient.info/forums/discuss/genital-he...,Ever since I suspected genital herpes (GH) and...,FelizCastus,2021-03-16T20:41+00:00,Most individuals experience symptoms within th...
4,Genital Herpes: Useful info for those looking ...,FelizCastus,535,2020-07-09T14:38+00:00,https://patient.info/forums/discuss/genital-he...,Ever since I suspected genital herpes (GH) and...,josiphine24649,2015-12-10T17:09+00:00,Do HSV1 & 2 infections look the same on the ge...


In [ ]:
len(df_final)

50595

In [ ]:
df_final.to_csv('gential_herpes_simplex_data_v1.csv', index=False)

In [ ]:
len(df_final)